In [1]:
from globalgatecompiler import *
import qiskit

# <U>Overview of Compiler Pipeline</U>

An overview of our compiler pipeline is shown in Fig. 1 and described in Sec. III in our paper.

 
#### COMPILER STEPS (described in more detail in sections below)   

INPUT: Qiskit circuit in any arbitrary gate set + hardware-level parameters + connectivity graph.  
OUTPUT: Qiskit circuit in neutral atom gate set {Rz($\lambda$), GR($\theta$,$\phi$), CZ, CCZ}, where these gates are defined in Eq. 1-4 in our paper.
     
1. Route circuit (using previous existing compiler methods, such as those available through Qiskit or Cirq).<br>

1. Transpile to gate set {U3($\theta$,$\phi$,$\lambda$),CZ,CCZ}; apply pre-compiling optimizations using existing transpiler passes.<br> 

1. Schedule into Single-Qubit Gate Moments (SQGMs, consisting of U3 gates) and Multi-Qubit Gate Moments (MQGMs, consisting of CZ and CCZ gates). This can either be done via `get_sifted_schedule` which minimizes the number of moments in the circuit in linear compile time, or via `get_theta_opt_circuit` which minimizes both number of moments and total GR rotation amount, at the cost of longer compile time.<br>

1. Decompose into neutral atom native gate set {Rz($\lambda$), GR($\theta$,$\phi$), CZ, CCZ}. This is done using `decompose_to_neutral_atom_gate_set`, with decomposition type specified as either `axial` (each SQGM is decomposed using a GR($\pi/2$,0) and GR($-\pi/2$,0) gate) or `transverse` (GR parameters are chosen such that global rotation angle is minimized for the given SQGM). <br>

Following compilation: Calculate fidelity and duration of the final circuit (`get_time_cost_data` and `get_fidelity_cost_data`), accounting for hardware-level parallelism constraints of CZ and CCZ gates. <br>

# <u>Compiler Steps Explained In More Detail</u>

## 1. Define Inputs to the Compiler

### A. Original Circuit [any universal gate set]

The input into our compiler is a QASM circuit (specifically, a Qiskit [QuantumCircuit](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.QuantumCircuit) object) in any arbitrary universal gate set. Here, as an example throughout this notebook, we use the 10-qubit QFT Adder circuit from ref. [1]. 

In [2]:
orig_circuit = qiskit.QuantumCircuit.from_qasm_file('example_circuit.qasm')
orig_circuit.draw(fold=-1)

┌───┐┌───────────┐     ┌───┐┌──────────┐     ┌───┐┌───────────┐     ┌───┐ ┌──────────┐           ┌───┐┌──────────┐┌───┐┌─────────┐┌───┐┌──────────┐┌───┐┌───────────┐  ┌─────────┐                                      ┌───┐          ┌─────────┐                                                                    ┌───┐┌──────────┐                                                    ┌───┐┌─────────────┐                                                   ┌───┐┌──────────────┐                                                     ┌───┐┌─────────────┐                                                   ┌───┐┌──────────────┐                                                     ┌───┐┌─────────────┐                                   ┌───┐┌──────────────┐                                     ┌───┐┌─────────────┐               ┌───┐┌──────────────┐     ┌───┐                  ┌───┐      ┌─────────┐ ┌───────┐┌───┐┌─────────┐┌───┐┌──────────┐         ┌───┐┌─────────┐┌───┐┌──────────┐     ┌───┐┌──────────┐     ┌───┐┌───────────┐              ┌───┐┌──────────┐     ┌───┐┌───────────┐                                                                                                                                 
q_0: ───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/32) ├─────┤ X ├┤ Rz(π/32) ├─────┤ X ├┤ Rz(-π/16) ├─────┤ X ├─┤ Rz(π/16) ├───────────┤ X ├┤ Rz(-π/8) ├┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/4) ├┤ X ├┤ Rz(-3π/4) ├──┤ Ry(π/2) ├──────────────■───────────────────────┤ X ├───────■──┤ Rz(π/2) ├────────────────────────────────────────────────────────────────────┤ X ├┤ Rz(-π/2) ├────────────────────────────────────────────────────┤ X ├┤ Rz(0.88623) ├───────────────────────────────────────────────────┤ X ├┤ Rz(-0.88623) ├─────────────────────────────────────────────────────┤ X ├┤ Rz(0.66567) ├───────────────────────────────────────────────────┤ X ├┤ Rz(-0.66567) ├─────────────────────────────────────────────────────┤ X ├┤ Rz(0.57692) ├───────────────────────────────────┤ X ├┤ Rz(-0.57692) ├─────────────────────────────────────┤ X ├┤ Rz(0.53708) ├───────────────┤ X ├┤ Rz(-0.53708) ├─────┤ X ├──■───────────────┤ X ├──■───┤ Ry(π/2) ├─┤ Rx(π) ├┤ X ├┤ Rz(π/4) ├┤ X ├┤ Rz(-π/4) ├─────────┤ X ├┤ Rz(π/8) ├┤ X ├┤ Rz(-π/8) ├─────┤ X ├┤ Rz(π/16) ├─────┤ X ├┤ Rz(-π/16) ├──────────────┤ X ├┤ Rz(π/32) ├─────┤ X ├┤ Rz(-π/32) ├─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
                                                                                                                   ┌───┐┌───────────┐     ┌───┐ ┌──────────┐           ┌───┐└─┬─┘└┬──────────┤┌───┐└─┬─┘├─────────┬┘┌───┐└─┬─┘└┬──────────┤┌───┐└─┬─┘┌┴──────────┤┌─────────┐└─┬─┘├─────────┬┘└─┬─┘└─────────┘└─┬─┘└──────────┘└─┬─┘└───────────┘  └─────────┘  ┌───┐       │         ┌─────────┐   └─┬─┘       │  └──┬───┬──┘  ┌──────────┐                  ┌───┐┌─────────────┐                └─┬─┘└──────────┘               ┌───┐┌──────────────┐                └─┬─┘└─────────────┘                ┌───┐┌─────────────┐               └─┬─┘└──────────────┘                ┌───┐┌──────────────┐                └─┬─┘└─────────────┘                ┌───┐┌─────────────┐               └─┬─┘└──────────────┘                ┌───┐┌──────────────┐                └─┬─┘└─────────────┘┌───┐┌─────────────┐               └─┬─┘└──────────────┘┌───┐┌──────────────┐                └─┬─┘└─────────────┘┌───┐          └─┬─┘└────┬───┬─────┘     └─┬─┘  │   ┌──────────┐└─┬─┘  │   └─────────┘ └───────┘└─┬─┘└─────────┘└─┬─┘├─────────┬┘┌───────┐└─┬─┘└─────────┘└─┬─┘└──────────┘┌───┐└─┬─┘├─────────┬┘┌───┐└─┬─┘└┬──────────┤         ┌───┐└─┬─┘├─────────┬┘┌───┐└─┬─┘└┬──────────┤     ┌───┐┌──────────┐     ┌───┐┌───────────┐                                                                                    
q_1: ─────────────────────────────────────────────

### B. Relevant Hardware-Level Parameters

The following hardware-level parameters must be defined:

* <u>Blockade Radius</u>: With neutral atoms, a multi-qubit gate can execute if and only if all qubits on which it acts are within a blockade radius of each other. Typical values for current architectures are ~5-15 micrometers [2], though this value may increase as experimental work progresses. 
* <u>Atom Spacing</u>: Distance between neighboring atom sites within an atom array. This should be larger than ~2-3 micrometers to avoid non-negligible crosstalk [3].
* <u>Rz($\pi$) duration</u>: Time required to execute an Rz gate of rotation $\pi$. For Rz gates with arbitrary angle, duration scales with that rotation angle: $t_{Rz(\lambda)}=t_{Rz(\pi)}*(\lambda/pi)$, where $t_{Rz(\lambda)}$ and $t_{Rz(\pi)}$ are the times required to execute Rz($\lambda$) and Rz($\pi$) gates, respectively.
* <u>GR($\pi$,$\phi$) duration</u>: Time required to execute a GR gate of rotation amount $\pi$. As above, duration of GR gates will scale with rotation angle: $t_{GR(\theta,\phi)}=t_{GR(\theta,\phi)}*(\theta/pi)$.
* <u>CZ duration</u> and <u>CCZ duration</u>: time required to execute a CZ gate and CCZ gate, respectively.
* Rz Constant: As explained in Sec. VI of our paper, we model fidelity of an Rz($\lambda$) gate as $F=1-C_{Rz}*\lambda$. The value for $C_{Rz}$ is chosen to be consistent with recent experimental work, and this is input as a parameter into `get_fidelity_cost_data`, which calculates the fidelity of the final circuit.
* <u>GR Constant</u>: We model fidelity of a GR($\theta$,$\phi$) gate as $F=1-C_{GR}*\theta^2$, again with $C_{GR}$ chosen to be consistent with experimental work and input as a parameter when calculating fidelity costs. 
* <u>CZ fidelity</u>, <u>CCZ fidelity</u> and <u>$T_2^*$ time</u> for the given device.

The numbers we use here are from references [3], [4], [5].

In [3]:
# used in constructing device connectivity graph
blockade_radius = 7 # micrometers
spacing = 3 # micrometers

# used in calculating circuit duration
rz_pi_duration = 0.167 # microseconds, Rz gate durations scaled by rotation amount
gr_pi_duration = 6.5 # microseconds, GR gate durations scaled by rotation amount
cz_duration = 0.270 # microseconds
ccz_duration = 0.389 # microseconds

# used in calculating circuit fidelity
rz_constant = 1.210e-4
gr_constant = 6.617e-5 
cz_fidelity = 0.995
ccz_fidelity = 0.979
t2_star = 4000 # microseconds

### C. Device Connectivity Graph

The connectivity graph is an undirected graph that captures which pairs of atoms can interact via multi-qubit gates. Nodes in the graph represent atoms, and edges exist between pairs of atoms that are within a blockade radius of each other. To execute a $CZ(q_a,q_b)$ gate, $(M(q_a),M(q_b))$ must be in the connectivity graph's edge set, where $M(q)$ is the hardware qubit (i.e., atom) to which the program qubit $q$ is mapped. To execute a $CCZ(q_a,q_b,q_c)$ gate, the pairs $(M(q_a),M(q_b))$, $(M(q_b),M(q_c))$, and $(M(q_a),M(q_c))$ must all be in the connectivity graph's edge set.

For given values of blockade radius, atom spacing, and input circuit size (n), `device_connectivity_graph` constructs a connectivity graph for an atom array arranged in a rectangular, grid-like structure. The rectangle's dimensions can be specified by parameters `grid_width` and `grid_height`. If these are left unspecified, dimensions will be $w=$ceil$(\sqrt{n})$ and $h=$ceil$(n/w)$. We choose a rectangular geometry because it works well with the constraints imposed by AODs used in neutral atom architectures [4]. However, arbitrary geometries are possible with neutral atom arrays [6], and our compiler code works with connectivity graphs corresponding to any arbitrary geometry. 

In [4]:
cg = device_connectivity_graph(orig_circuit,blockade_radius,spacing)

cg.nodes, cg.edges

(NodeView((0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11)),
 EdgeView([(0, 1), (0, 2), (0, 4), (0, 5), (0, 6), (0, 8), (0, 9), (1, 2), (1, 3), (1, 4), (1, 5), (1, 6), (1, 7), (1, 8), (1, 9), (1, 10), (2, 3), (2, 4), (2, 5), (2, 6), (2, 7), (2, 9), (2, 10), (2, 11), (3, 5), (3, 6), (3, 7), (3, 10), (3, 11), (4, 5), (4, 6), (4, 8), (4, 9), (4, 10), (5, 6), (5, 7), (5, 8), (5, 9), (5, 10), (5, 11), (6, 7), (6, 8), (6, 9), (6, 10), (6, 11), (7, 9), (7, 10), (7, 11), (8, 9), (8, 10), (9, 10), (9, 11), (10, 11)]))

## 2. Route Circuit

For mapping and routing, methods from existing prior works are sufficient. Here, we use the [Qiskit's SabreSwap implementation](https://docs.quantum.ibm.com/api/qiskit/qiskit.transpiler.passes.SabreSwap), based on ref. [7]. 

Note that the majority of mapping and routing methods assume a gate set consisting only of single-qubit and two-qubit gates. Neutral atom architectures, however, support native multi-qubit gates; if the original input circuit contains three-qubit gates such as Toffolis, these can remain in the circuit (these are converted to CCZ gates in the following step). For these circuits, we use the approaches from Ref. [8], which generalizes mapping and routing methods to work with three-qubit gates.

Importantly, mapping and routing should occur prior to the other steps in the compiler, so that optimizations in the later compiler passes account for the SWAP gates that are inserted during routing.

In [5]:
# get coupling map from connectivity graph
coupling_list = [(edge[0],edge[1]) for edge in cg.edges] + [(edge[1],edge[0]) for edge in cg.edges]
coupling_map = qiskit.transpiler.CouplingMap(coupling_list)

# find initial layout (initial mapping)
layout_object = qiskit.transpiler.passes.SabreLayout(coupling_map)
dag = layout_object.run(qiskit.converters.circuit_to_dag(orig_circuit))

# route circuit (insert SWAPs)
routed_dag = qiskit.transpiler.passes.SabreSwap(coupling_map).run(dag)
routed_circuit = qiskit.converters.dag_to_circuit(routed_dag)

n = len(routed_circuit.qubits)

routed_circuit.draw(fold=-1)

q_0: ─────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■────────────────────────■────────────────────────■─────────────────────────■────────────────────────■──────────────────────────■────────────────────────■──────────────────────────■────────────────────────■──────────────────────────■────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
      ┌──────────┐┌───────────┐                       ┌─────────┐                             ┌──────────┐                        ┌──────────┐                                                                                                                                                                                                                            ┌───┐                               ┌───┐┌─────────┐   ┌─┴─┐      ┌──────────┐  ┌─┴─┐┌─────────────┐     ┌─┴─┐     ┌──────────────┐┌─┴─┐┌─────────────┐     ┌─┴─┐      ┌──────────────┐┌─┴─┐┌─────────────┐     ┌─┴─┐      ┌──────────────┐┌─┴─┐┌─────────────┐     ┌─┴─┐      ┌──────────────┐┌─┴─┐                                                                                                                                                                  ┌───┐                       ┌───┐┌───────────┐                                                                                                                                                           ┌───────────┐                       ┌──────────┐                         ┌──────────┐                     ┌─────────┐ ┌───────┐
 q_1: ┤ Ry(-π/2) ├┤ Rz(-3π/4) ├──■────────────────■───┤ Rz(π/8) ├──────■───────────────────■──┤ Rz(π/16) ├──■─────────────────■───┤ Rz(π/32) ├──────────────────■──────────────────────■──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├─────────────■─────────────────┤ X ├┤ Rz(π/2) ├───┤ X ├──────┤ Rz(-π/2) ├──┤ X ├┤ Rz(0.88623) ├─────┤ X ├─────┤ Rz(-0.88623) ├┤ X ├┤ Rz(0.66567) ├─────┤ X ├──────┤ Rz(-0.66567) ├┤ X ├┤ Rz(0.57692) ├─────┤ X ├──────┤ Rz(-0.57692) ├┤ X ├┤ Rz(0.53708) ├─────┤ X ├──────┤ Rz(-0.53708) ├┤ X ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────┤ X ├───────■───────────────┤ X ├┤ Rz(-π/32) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─────────────────────■──┤ Rz(-π/16) ├──■────────────────■───┤ Rz(-π/8) ├────■─────────────────■──┤ Rz(-π/4) ├──■───────────────■──┤ Ry(π/2) ├─┤ Rx(π) ├
      └──────────┘└───────────┘  │                │   └─────────┘      │                   │  └──────────┘  │                 │   └──────────┘                  │                      │                                                                                                                                                                                  └─┬─┘             │                 └─┬─┘└─────────┘   └───┘      └──────────┘  └───┘└─────────────┘     └───┘     └──────────────┘└───┘└─────────────┘     └───┘      └──────────────┘└───┘└─────────────┘ 

## 3. Transpile to Standard Gate Set 
### [any gate set] &rarr; {U3($\theta$,$\phi$,$\lambda$), CZ, CCZ}

Before scheduling, the circuit must be transpiled into what we call the Standard Gate Set, consisting of {U3($\theta$,$\phi$,$\lambda$),CZ,CCZ}. 

If the original input circuit contains only 1-qubit and 2-qubit gates, this compiler step will translate the circuit into {U3,CZ}. Qiskit's `transpile` method with the `basis_gates` parameter set to `['u3','cz']`, as shown below, accomplishes this. It also applies various optimizations to the circuit, such as commuting and merging gates when possible. 

If the original circuit also contains 3-qubit gates, Qiskit's transpile pass will still convert to {U3,CZ}, even if the `basis_gates` parameter is set to `['u3','cz','ccz']`; i.e., any 3-qubit gates will be decomposed into 1-qubit and 2-qubit gates. We provide the function `transpile_to_u3_cz_ccz` which will keep 3-qubit gates in the circuit, converting to the gate set {U3,CZ,CCZ}, while also applying Qiskit's optimizations to blocks of 1-qubit and 2-qubit gates in between the 3-qubit gates.

In [6]:
standard_gate_set_circuit = qiskit.transpile(routed_circuit,basis_gates=['u3','cz'],optimization_level=3)
standard_gate_set_circuit.draw(fold=-1)

global phase: 5.6782
           ┌─────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                                         ┌──────────────────┐                    ┌────────────────────────┐                                                                                                                                                                                                                                                                                                                                                                                                                                                                  
 q_0: ─────┤ U3(π,0,π/2) ├───────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─┤ U3(π,-π/4,-3π/4) ├─────────■──────────┤ U3(0,0.48762,-0.48762) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────
          ┌┴─────────────┴─┐                                ┌─────────────┐                               ┌──────────────┐                                ┌──────────────┐                                      ┌─────────────┐                                                                                                                                                                                ┌─────────────┐                  ┌─────────────┐                          ┌─────────────────────┐ │ ├──────────────────┤         │          └─┬────────────────────┬─┘                                                          ┌─────────────┐      ┌─────────────┐      ┌──────────────────┐                                                                                                                                                                              ┌───────────────┐                               ┌──────────────┐                              ┌──────────────┐                             ┌──────────────┐ 
 q_1: ────┤ U3(π/2,π/4,-π) ├─────■──────────────────────■───┤ U3(0,0,π/8) ├────■──────────────────────■───┤ U3(0,0,π/16) ├───■───────────────────────■────┤ U3(0,0,π/32) ├──────■──────────────────────────■────┤ U3(π/2,0,π) ├──────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■─┤ U3(π/2,0,π) ├────────────────■─┤ U3(π/2,0,π) ├────────────────────────■─┤ U3(1.8203,π/2,-π/2) ├─■─┤ U3(1.0951,0,π/2) ├─────────■────────────┤ U3(2.9154,-π/2,-π) ├──────────────────────────────────────────────────────────■─┤ U3(π/2,0,π) ├────■─┤ U3(π/2,0,π) ├────■─┤ U3(π/2,-π/32,-π) ├────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────────■──────────────────────────■──┤ U3(0,0,-π/16) ├───■───────────────────────■───┤ U3(0,0,-π/8) ├───■────

## 4. Schedule Circuit Into SQGMs and MQGMs

Before decomposition into global gates can occur, the circuit must be scheduled into Single-Qubit Gate Moments and Multi-Qubit Gate Moments. Each SQGM consists of a subset of U3 gates that can all be executed in parallel. MQGMs consist of the CZ and CCZ gates in between consecutive SQGMs; these CZs and CCZs may or may not be executable in parallel. Parallelism constraints on multi-qubit gates are imposed at a later step.

We provide two approaches for scheduling, explained further below. The Sifting approach maximizes parallelism of single-qubit gates, thus minimizing the number of SQGMs, and therefore minimizing the *number* of GR gates required when decomposing the circuit. This requires compile time that is linear in the number of gates in the circuit. The $\theta$-Opt approach not only minimizes the number of GR gates, but also the total GR rotation *amount* in the final decomposed circuit (assuming that the transverse decomposition method is used). The motivation behind this optimization is that both duration and fidelity of a given gate depends on its rotation angle, and GR costs dominate gate execution costs on many neutral atom architectures. However, this optimization comes at a cost of longer compile time, compared to the Sifting approach.

In both cases, we represent a schedule as a list of moments, where each moment is a list of Qiskit [CircuitInstruction](https://docs.quantum.ibm.com/api/qiskit/qiskit.circuit.CircuitInstruction) objects. 

### Sifting Scheduling

As described in Sec. V-B, "sifting" through a circuit relies on a defined indicator function which maps each gate in the DAG's node set to either 0 or 1. Gates mapped to 0 are not placed within the same moment as gates mapped to 1, and parallelism is maximized for the gates mapped to 1. The `get_sifted_schedule` allows for arbitrary indicator functions. In this work, we use `indicator_fn=lambda op: len(op[1])==1`; i.e., single-qubit gates mapped to 1 and all other gates (here, all multi-qubit gates) are mapped to 0. Since we assume the circuit has been transpiled into {U3,CZ,CCZ}, this means each moment consists of only U3 gates or only CZ and CCZ gates, with parallelism of U3 gates maximized. 

In [7]:
sifted_schedule = get_sifted_schedule(standard_gate_set_circuit,indicator_fn=lambda op: len(op[1])==1)
sifted_schedule

[[CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[3.141592653589793, 0.0, 1.5707963267948966]), qubits=(Qubit(QuantumRegister(12, 'q'), 0),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[1.5707963267948963, 0.7853981633974483, -3.141592653589793]), qubits=(Qubit(QuantumRegister(12, 'q'), 1),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[3.1415926535897927, -1.5158476654564725, 0.05494866133842402]), qubits=(Qubit(QuantumRegister(12, 'q'), 2),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[1.5707963267948966, 0, 3.141592653589793]), qubits=(Qubit(QuantumRegister(12, 'q'), 3),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[6.4736570491389385e-16, -1.8157749899217608, -2.896613990462929]), qubits=(Qubit(QuantumRegister(12, '

### $\theta$-Opt Scheduling

The $\theta$-Opt scheduling algorithm, described in Sec. V-C, uses a dynamic programming approach to schedule gates in a way that minimizes the GR rotation amount required when decomposed using the transverse decomposition (in the next compiler step). In the transverse decomposition, each SQGM is decomposed using two GR gates of rotation angle $\pm\frac{1}{2}\theta_{max}$, where $\theta_{max}$ is the maximum $\theta$ Euler angle parameter of any U3 gate in that SQGM. Therefore, formally, we minimize the objective $\sum_{M}$max$(g.\theta|g\in M)$, where the sum occurs over all SQGMs and $g.\theta$ is the $\theta$ Euler angle parameter of gate $g$. 

In [8]:
theta_opt_schedule = get_theta_opt_schedule(standard_gate_set_circuit)
theta_opt_schedule

[[CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[3.141592653589793, 0.0, 1.5707963267948966]), qubits=(Qubit(QuantumRegister(12, 'q'), 0),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[3.1415926535897927, -1.5158476654564725, 0.05494866133842402]), qubits=(Qubit(QuantumRegister(12, 'q'), 2),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[1.5707963267948966, 0, 3.141592653589793]), qubits=(Qubit(QuantumRegister(12, 'q'), 3),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[1.5707963267948966, 0, 3.141592653589793]), qubits=(Qubit(QuantumRegister(12, 'q'), 5),), clbits=()),
  CircuitInstruction(operation=Instruction(name='u3', num_qubits=1, num_clbits=0, params=[1.5707963267948966, 0, 3.141592653589793]), qubits=(Qubit(QuantumRegister(12, 'q'), 6),), clbits=()),
  CircuitInstructi

### Obtaining Schedule Information

The following functions are not used in the compiler pipeline, but are rather intended to help in obtaining information about a given schedule and in understanding how the gates are scheduled:
* `get_schedule_GR_cost`: For a given schedule, this calculates what the total GR rotation amount will be in the final circuit, once the transverse decomposition has been applied to conver the circuit into gate set {Rz,GR,CZ,CCZ}. Smaller total rotation amount is better.  
* `print_schedule`: prints the schedule in a format that is easier to read. If `with_angles` is set to `False`, this prints the gate type (U3, CZ, or CCZ) and qubit operand(s) for each gate, with gates grouped by moment. If `with_angles` is set to `True`, the Euler angles $\theta$, $\phi$, and $\lambda$ are also included.
* `get_circuit_from_schedule_with_barriers`: returns the circuit with barriers between adjacent moments. Note that this circuit is not used as part of the compiler pipeline, and the barrier gates here are not intended to indicate pieces of the circuit to be optimized separately (as is their usual purpose); rather, the barriers are used simply for visualizing which gates are scheduled into the same moment. 

In [9]:
get_schedule_GR_cost(sifted_schedule)

49.254

In [10]:
get_schedule_GR_cost(theta_opt_schedule)

35.691

In [11]:
print_schedule(theta_opt_schedule,with_angles=False)

[U3(q0), U3(q2), U3(q3), U3(q5), U3(q6), U3(q7), U3(q9), U3(q10), U3(q1), U3(q4)]
[CZ(q1,q6)]
[U3(q6)]
[CZ(q1,q6)]
[U3(q1)]
[CZ(q1,q5)]
[U3(q6), U3(q5)]
[CZ(q1,q5)]
[U3(q1)]
[CZ(q1,q3)]
[U3(q5), U3(q3)]
[CZ(q6,q5), CZ(q1,q3)]
[U3(q5), U3(q3), U3(q1)]
[CZ(q6,q5), CZ(q1,q7)]
[U3(q6)]
[CZ(q6,q3)]
[U3(q5), U3(q3), U3(q7)]
[CZ(q1,q7), CZ(q6,q3)]
[U3(q7), U3(q6)]
[CZ(q6,q7)]
[U3(q3), U3(q7)]
[CZ(q5,q3), CZ(q6,q7)]
[U3(q1), U3(q6), U3(q3), U3(q7)]
[CZ(q5,q3)]
[U3(q5)]
[CZ(q5,q7)]
[U3(q3), U3(q7)]
[CZ(q5,q7)]
[U3(q7)]
[CZ(q3,q7)]
[U3(q7)]
[CZ(q3,q7), CZ(q3,q6)]
[U3(q3), U3(q6), U3(q5), U3(q7)]
[CZ(q4,q5), CZ(q7,q1), CZ(q6,q3)]
[U3(q1), U3(q7), U3(q3), U3(q6), U3(q5)]
[CZ(q1,q7), CZ(q3,q6)]
[U3(q3)]
[CZ(q3,q10)]
[U3(q6), U3(q1), U3(q7), U3(q10)]
[CZ(q7,q1), CZ(q6,q9)]
[U3(q1), U3(q7), U3(q9)]
[CZ(q0,q1), CZ(q2,q7)]
[U3(q4), U3(q3), U3(q6), U3(q0), U3(q2), U3(q1), U3(q7)]
[CZ(q4,q5), CZ(q3,q10), CZ(q6,q9), CZ(q0,q1), CZ(q2,q7)]
[U3(q4), U3(q3), U3(q1), U3(q5), U3(q9), U3(q10), U3(q7), U3(q6), U3

In [12]:
get_circuit_from_schedule_with_barriers(theta_opt_schedule,n).draw(fold=-1)

┌─────────────┐       ░     ░                      ░     ░                 ░     ░                      ░     ░                  ░     ░                       ░     ░                       ░        ░                 ░     ░                       ░        ░                       ░     ░                       ░     ░                       ░     ░                 ░     ░                      ░     ░                      ░     ░                      ░        ░                        ░           ░                      ░        ░                         ░     ░                       ░        ░                         ░     ░ ┌──────────────────┐ ░           ░ ┌────────────────────────┐ ░        ░                 ░        ░                 ░        ░                  ░     ░                      ░     ░                      ░     ░                      ░     ░                  ░     ░                      ░     ░                       ░     ░                  ░     ░                       ░        ░                  ░     ░                       ░        ░                   ░     ░                       ░     ░                  ░     ░                      ░     ░                  ░     ░                      ░     ░                      ░ 
 q_0: ─────┤ U3(π,0,π/2) ├───────░─────░──────────────────────░─────░─────────────────░─────░──────────────────────░─────░──────────────────░─────░───────────────────────░─────░───────────────────────░────────░─────────────────░─────░───────────────────────░────────░───────────────────────░─────░───────────────────────░─────░───────────────────────░─────░─────────────────░─────░──────────────────────░─────░──────────────────────░─────░──────────────────────░────────░────────────────────────░───────────░──────────────────────░────────░─────────────────────────░─────░───────────────────────░────────░─────────────────────────░──■──░─┤ U3(π,-π/4,-3π/4) ├─░──■────────░─┤ U3(0,0.48762,-0.48762) ├─░────────░─────────────────░────────░─────────────────░────────░──────────────────░─────░──────────────────────░─────░──────────────────────░─────░──────────────────────░─────░──────────────────░─────░──────────────────────░─────░───────────────────────░─────░──────────────────░─────░───────────────────────░────────░──────────────────░─────░───────────────────────░────────░───────────────────░─────░───────────────────────░─────░──────────────────░─────░──────────────────────░─────░──────────────────░─────░──────────────────────░─────░──────────────────────░─
          ┌┴─────────────┴─┐     ░     ░                      ░     ░ ┌─────────────┐ ░     ░                      ░     ░ ┌──────────────┐ ░     ░                       ░     ░    ┌──────────────┐   ░        ░                 ░     ░                       ░        ░                       ░     ░                       ░     ░    ┌─────────────┐    ░     ░                 ░     ░                      ░     ░                      ░     ░                      ░        ░                        ░           ░   ┌─────────────┐    ░        ░                         ░     ░    ┌─────────────┐    ░        ░ ┌─────────────────────┐ ░  │  ░ ├──────────────────┤ ░  │        ░ └─┬────────────────────┬─┘ ░        ░ ┌─────────────┐ ░        ░ ┌─────────────┐ ░        ░                  ░     ░ ┌──────────────────┐ ░     ░                      ░     ░                      ░     ░                  ░     ░                      ░     ░                       ░     ░                  ░     ░                       ░        ░                  ░     ░                       ░        ░ ┌───────────────┐ ░     ░                       ░     ░ ┌──────────────┐ ░     ░                      ░     ░ ┌──────────────┐ ░     ░                      ░     ░   ┌──────────────┐   ░ 
 q_1: ────┤ U3(π/2,π/4,-π) ├─────░──■──░──────────────────────░──■──░─┤ U3(0,0,π/8) ├─░──■──░──────────────────────░──■──░─┤ U3(0,0,π/16) ├─░──■──░───────────────────────░──■──░────┤ U3(0,0,π/32) ├───░─────■──░──────

## 5. Decompose Into Neutral Atom Gate Set
### {U3($\theta$,$\phi$,$\lambda$), CZ, CCZ} &rarr; {Rz($\lambda$), GR($\theta$,$\phi$), CZ, CCZ}

After scheduling, each MQGM is already in the Neutral Atom Gate Set. Each SQGM, however, must be decomposed into Rz and GR gates. The `decomposition_type` parameter can be set either the `axial` or `transverse`. If not specified, it defaults to `transverse`. With the axial decomposition, each set of parallel U3 gates is decomposed using a GR($\pi/2$,0) gate and a GR($-\pi/2$,0) gate. This decomposition maintains the optimized parallelism of single-qubit gates that was achieved during scheduling; however, it does not optimize GR rotation amount. The transverse decomposition chooses the GR parameters $\theta$ and $\phi$ such that, for the given SQGM, the minimum possible GR rotation amount is used. See Sec. IV in our paper for more details.

If `use_backlog` is set to `True`, the last column of Rz gates (following the second GR gate in the decomposed moment) is commuted past the following MQGM and merged with the first column of Rz gates in the next decomposed SQGM, as described in Sec. IV-D. We note that this optimization of Rz gates should always be used, as it does not increase compilation time nor costs of other gate types. We provide the option to set `use_backlog` to `False`, however, as doing so can make it easier to understand and visualize the decomposition. As also mentioned in Sec. IV-D, the parameters $\eta$, $|\theta_{max}|$, and $\sigma_j$ can be adjusted to further optimize Rz costs. If not specified, these are set to `eta=0`, `sign_theta_max=1`, and `sign_sigma_j=1`.

In [13]:
# decompose using the axial decomposition, without "backlog"
decomposed_circ_ax,decomposed_moments_ax = decompose_to_neutral_atom_gate_set(theta_opt_schedule,
                                                                              len(routed_circuit.qubits),
                                                                              decomposition_type='axial',
                                                                              use_backlog=False)
decomposed_circ_ax.draw(fold=-1)

┌─────────┐   ┌─────────────┐ ┌───────┐ ┌──────────────┐                              ┌─────────────┐           ┌──────────────┐                         ┌─────────────┐┌──────────────┐                       ┌─────────────┐           ┌──────────────┐                           ┌─────────────┐┌──────────────┐                       ┌─────────────┐            ┌──────────────┐                           ┌─────────────┐            ┌──────────────┐                             ┌─────────────┐┌──────────────┐                        ┌─────────────┐            ┌──────────────┐                                      ┌─────────────┐            ┌──────────────┐                           ┌─────────────┐            ┌──────────────┐                           ┌─────────────┐            ┌──────────────┐                          ┌─────────────┐┌──────────────┐                       ┌─────────────┐           ┌──────────────┐                          ┌─────────────┐           ┌──────────────┐                           ┌─────────────┐           ┌──────────────┐                                      ┌─────────────┐           ┌──────────────┐                                      ┌─────────────┐              ┌──────────────┐                    ┌─────────────┐┌──────────────┐                            ┌─────────────┐              ┌──────────────┐                                           ┌─────────────┐              ┌──────────────┐                        ┌───────────┐┌─────────────┐  ┌───────┐   ┌──────────────┐┌──────────┐   ┌──────────────┐                            ┌─────────────┐              ┌──────────────┐┌─────────────┐               ┌─────────────┐           ┌──────────────┐               ┌─────────────┐           ┌──────────────┐                  ┌─────────────┐┌──────────────┐                        ┌─────────────┐           ┌──────────────┐                            ┌─────────────┐           ┌──────────────┐                         ┌─────────────┐           ┌──────────────┐                           ┌─────────────┐┌──────────────┐                       ┌─────────────┐           ┌──────────────┐                           ┌─────────────┐            ┌──────────────┐                           ┌─────────────┐┌──────────────┐                       ┌─────────────┐            ┌──────────────┐                              ┌─────────────┐┌──────────────┐                       ┌─────────────┐            ┌──────────────┐                                      ┌─────────────┐┌──────────────┐                       ┌─────────────┐            ┌──────────────┐                           ┌─────────────┐┌──────────────┐              ┌─────────────┐           ┌──────────────┐                           ┌─────────────┐┌──────────────┐              ┌─────────────┐           ┌──────────────┐                         ┌─────────────┐           ┌──────────────┐             
 q9_0: ──┤ Rz(π/2) ├───┤0            ├─┤ Rz(π) ├─┤0             ├──────────────────────────────┤0            ├───────────┤0             ├─────────────────────────┤0            ├┤0             ├───────────────────────┤0            ├───────────┤0             ├───────────────────────────┤0            ├┤0             ├───────────────────────┤0            ├────────────┤0             ├───────────────────────────┤0            ├────────────┤0             ├─────────────────────────────┤0            ├┤0             ├────────────────────────┤0            ├────────────┤0             ├──────────────────────────────────────┤0            ├────────────┤0             ├───────────────────────────┤0            ├────────────┤0             ├───────────────────────────┤0            ├────────────┤0             ├──────────────────────────┤0            ├┤0             ├───────────────────────┤0            ├───────────┤0             ├──────────────────────────┤0            ├───────────┤0             ├───────────────────────────┤0            ├───────────┤0             ├──────────────────────────────────────┤0            ├────────

In [14]:
# decompose using transverse decomposition, with "backlog"
decomposed_circ_tr,decomposed_moments_tr = decompose_to_neutral_atom_gate_set(theta_opt_schedule,
                                                                              len(routed_circuit.qubits),
                                                                              decomposition_type='transverse',
                                                                              use_backlog=True)
decomposed_circ_tr.draw(fold=-1)

┌──────────┐ ┌────────────────┐ ┌────────┐ ┌───────────────┐   ┌────────────────┐          ┌───────────────┐                     ┌────────────────┐               ┌───────────────┐                                 ┌─────────────────┐               ┌────────────────┐                  ┌────────────────┐                ┌───────────────┐                                    ┌────────────────┐                ┌───────────────┐                               ┌─────────────────┐          ┌────────────────┐                              ┌─────────────────┐               ┌────────────────┐                  ┌────────────────┐                ┌───────────────┐                                ┌────────────────┐               ┌───────────────┐                  ┌─────────────────┐          ┌────────────────┐   ┌────────────────┐          ┌───────────────┐                                 ┌────────────────┐               ┌───────────────┐                                    ┌────────────────┐              ┌───────────────┐                         ┌────────────────┐              ┌───────────────┐                             ┌────────────────────┐               ┌───────────────────┐     ┌─────────┐   ┌────────────────┐   ┌────────┐  ┌───────────────┐   ┌─────────────┐                               ┌────────────────┐               ┌───────────────┐                     ┌────────────────┐          ┌───────────────┐      ┌────────────────┐          ┌───────────────┐                        ┌────────────────┐               ┌───────────────┐                  ┌────────────────┐          ┌───────────────┐   ┌─────────────────┐          ┌────────────────┐                  ┌────────────────┐               ┌───────────────┐                  ┌────────────────┐                ┌───────────────┐                  ┌─────────────────┐               ┌────────────────┐                                 ┌────────────────┐                ┌───────────────┐                     ┌────────────────┐                ┌───────────────┐      ┌─────────────────┐          ┌────────────────┐                     ┌────────────────┐          ┌───────────────┐                              ┌────────────────┐          ┌───────────────┐┌─────────────┐
 q10_0: ──┤ Rz(-π/2) ├─┤0               ├─┤ Rz(-π) ├─┤0              ├───┤0               ├──────────┤0              ├─────────────────────┤0               ├───────────────┤0              ├─────────────────────────────────┤0                ├───────────────┤0               ├──────────────────┤0               ├────────────────┤0              ├────────────────────────────────────┤0               ├────────────────┤0              ├───────────────────────────────┤0                ├──────────┤0               ├──────────────────────────────┤0                ├───────────────┤0               ├──────────────────┤0               ├────────────────┤0              ├────────────────────────────────┤0               ├───────────────┤0              ├──────────────────┤0                ├──────────┤0               ├───┤0               ├──────────┤0              ├─────────────────────────────────┤0               ├───────────────┤0              ├────────────────────────────────────┤0               ├──────────────┤0              ├─────────────────────────┤0               ├──────────────┤0              ├─────────────────────────────┤0                   ├───────────────┤0                  ├─■───┤ Rz(π/4) ├───┤0               ├───┤ Rz(-π) ├──┤0              ├─■─┤ Rz(0.29778) ├───────────────────────────────┤0               ├───────────────┤0              ├─────────────────────┤0               ├──────────┤0              ├──────┤0               ├──────────┤0              ├────────────────────────┤0               ├───────────────┤0              ├──────────────────┤0               ├──────────┤0              ├───┤0                ├──────────┤0               ├──────────────────┤0               ├───────────────┤0              ├──────────────────┤0               ├────────────────┤0     

## 6. Calculate Fidelity and Duration of Final Circuit

Calculations of the final circuit's total duration and fidelity are explained in Sec. VI-A and Sec. VI-B, respectively. We note, importantly, that the calculations of each MQGM's duration accounts for parallelism constraints at both the program and hardware level. Specifically:
* <u>Program-level parallelism constraints</u>: Two different gates cannot execute simultaneously on the same qubit. Formally, for $CZ(q_a,q_b)$ and $CZ(q_c,q_d)$ to execute in parallel, we must have $\{q_a,q_b\}\cap\{q_c,q_d\}=\emptyset$, and similarly for CCZ gates. Otherwise, these execute in series.
* <u>Hardware-level parallelism constraints</u>: For a multi-qubit gate (i.e., CZ or CCZ) to execute simultaneously with another gate, operands of *different* gates cannot be within a blockade radius of each other. Formally, $CZ(q_a,q_b)$ and $CZ(q_c,q_d)$ to execute in parallel, the edges $(q_a,q_c)$, $(q_a,q_d)$, $(q_b,q_c)$, and $(q_b,q_d)$ *cannot* be in the edge set of the connectivity graph, and similarly for CCZ gates.

The `duration` field of each moment object is set by the `set_duration` method, which is called either by `get_time_cost_data` or `get_fidelity_cost_data` (whichever is run first). If `get_fidelity_cost_data` is run prior to `get_time_cost_data`, the parameters relating to gate duration (i.e., `rz_pi_duration`, `gr_pi_duration`, `cz_duration`, and `ccz_duration`) must also be passed as inputs into the function so that moments' durations can be calculated; the durations of each moment are used in determining the circuit's idle errors. 

### A. Circuit Duration

The output of the `get_time_cost_data` is a dict that specifies time spent executing Rz gates, time spent executing GR gates, and time spent executing multi-qubit gates (CZ and/or CCZ; since these two gate types can be executed in parallel with each other, separately calculating time spent on each does not make sense). The circuit's total duration is equivalent to the sum of these. 

In [15]:
get_time_cost_data(decomposed_moments_tr,rz_pi_duration,gr_pi_duration,cz_duration,ccz_duration,cg) # microseconds

{'rz': 8.3, 'gr': 73.844, 'multi': 11.34, 'total': 93.485}

### B. Fidelity

The output of the `get_fidelity_cost_data` is a dict that specifies the gate errors (i.e., those that occur from applications of gates; due to, e.g., imperfections in the microwave pulses, inhomogeneity of Rabi frequency in the array, etc.) and idle errors (i.e., those that occur when qubits are idle; due to decoherence as a result of unwanted interactions with the environment). The circuit's fidelity is equal to the product of these.

In [16]:
get_fidelity_cost_data(decomposed_moments_tr,rz_constant,gr_constant,cz_fidelity,ccz_fidelity,t2_star)

{'gate': 0.702, 'idle': 0.977, 'total': 0.685}

## <U>REFERENCES:<U> 
(Note that the reference numbers here do not correspond to the how the references in our paper are numbered)    
    
[1] A. L. Jonathan M Baker, Casey Duckering, “quantumcircuitbench- marks,” Dec. 2020. https://github.com/jmbaker94/quantumcircuitbenchmarks <br>
    
[2] S. Ebadi, A. Keesling, M. Cain, T. T. Wang, H. Levine, D. Bluvstein, G. Semeghini, A. Omran, J.-G. Liu, R. Samajdar, X.-Z. Luo, B. Nash, X. Gao, B. Barak, E. Farhi, S. Sachdev, N. Gemelke, L. Zhou, S. Choi, H. Pichler, S. T. Wang, M. Greiner, V. Vuletic, and M. D. Lukin, “Quantum optimization of maximum independent set using rydberg atom arrays,” Science, vol. 376, no. 6598, pp. 1209–1215, 2022. https://www.science.org/doi/abs/10.1126/science.abo6587 <br>
    
[3] T. M. Graham, Y. Song, J. Scott, C. Poole, L. Phuttitarn, K. Jooya, P. Eichler, X. Jiang, A. Marra, B. Grinkemeyer, M. Kwon, M. Ebert, J. Cherek, M. T. Lichtman, M. Gillette, J. Gilbert, D. Bowman, T. Ballance, C. Campbell, E. D. Dahl, O. Crawford, N. S. Blunt, B. Rogers, T. Noel, and M. Saffman, “Multi-qubit entanglement and algorithms on a neutral-atom quantum computer,” Nature, vol. 604, no. 7906, pp. 457–462, Apr. 2022, number: 7906 Publisher: Nature Publishing Group. https://www.nature.com/articles/s41586-022-04603-6 <br>
    
[4] D. Bluvstein, H. Levine, G. Semeghini, T. T. Wang, S. Ebadi, M. Kalinowski, A. Keesling, N. Maskara, H. Pichler, M. Greiner, V. Vuletic ́, and M. D. Lukin, “A quantum processor based on coherent transport of entangled atom arrays,” Nature, vol. 604, no. 7906, pp. 451–456, Apr. 2022, number: 7906 Publisher: Nature Publishing Group. https://www.nature.com/articles/s41586-022-04592-6 <5>
    
[5] S. J. Evered, D. Bluvstein, M. Kalinowski, S. Ebadi, T. Manovitz, H. Zhou, S. H. Li, A. A. Geim, T. T. Wang, N. Maskara, H. Levine, G. Semeghini, M. Greiner, V. Vuletic ́, and M. D. Lukin, “High-fidelity parallel entangling gates on a neutral-atom quantum computer,” Nature, vol. 622, no. 7982, pp. 268–272, Oct. 2023. https://www.nature.com/articles/s41586-023-06481-y <br>
    
[6] K. Singh, S. Anand, A. Pocklington, J. T. Kemp, and H. Bernien, “Dual-Element, Two-Dimensional Atom Array with Continuous-Mode Operation,” Physical Review X, vol. 12, no. 1, p. 011040, Mar. 2022, publisher: American Physical Society. https://link.aps.org/doi/10.1103/PhysRevX.12.011040
    
[7] G. Li, Y. Ding, X. Yuan, "Tackling the Qubit Mapping Problem for NISQ-Era Quantum Devices," May 2019, arXiv:1809.02573 [quant-ph]. http://arxiv.org/abs/1809.02573
    
[8] J. M. Baker, A. Litteken, C. Duckering, H. Hoffman, H. Bernien, and F. T. Chong, “Exploiting Long-Distance Interactions and Tolerating Atom Loss in Neutral Atom Quantum Architectures,” Nov. 2021, arXiv:2111.06469 [quant-ph]. http://arxiv.org/abs/2111.06469